In [340]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
from statistics import mean 
import matplotlib.pyplot as plt
import matplotlib
import pickle
import joblib
from IPython.display import display

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

from sklearn.cluster import KMeans
from scipy import stats
from sklearn.utils import shuffle


# Siamese Network Training

In [341]:
num_classes = 6
epochs = 20

In [342]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [343]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [344]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [345]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [346]:

def create_test_pairs(num,classes, x, y):
    np.random.seed(10)
    n = 500
    arr_x_test = np.empty((500,60,2,28,28))
    arr_y_test = np.empty((500,60,2))
    index_left = np.random.randint(0,500,10)
    index_right = np.random.randint(0,500,500)
    temp_arr1 = x[np.where(y==num)][index_right]
    temp_arr2 = y[np.where(y==num)][index_right]    
    for i in range(classes):
        arr_x_test[:,i*10:i*10+10,0]=x[np.where(y==i)][index_left]
        arr_y_test[:,i*10:i*10+10,0] = y[np.where(y==i)][index_left]
    for i in range(n):
        arr_x_test[i,:,1] = temp_arr1[i]
        arr_y_test[i,:,1] = temp_arr2[i]
    return arr_x_test, arr_y_test


In [347]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

In [348]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

In [349]:
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [350]:
def create_pred_matrix(classes, pairs, models):
    pred_matrix = np.zeros((classes, 500))
    for i in range(len(pairs)):
        k = 0
        for j in range(0,len(pairs[0]),10):
            pred_array = models.predict([pairs[i,j:j+10,0], pairs[i,j:j+10,1]])
            pred_avg = np.mean(pred_array)
            pred_matrix[k,i] = pred_avg
            k += 1
    return pred_matrix

In [351]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

In [352]:
# create training positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

In [353]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

In [354]:
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [355]:
# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs)

Epoch 1/20
65040/65040 [==============================] - 3s 50us/step - loss: 0.0794 - accuracy: 0.9160
Epoch 2/20
65040/65040 [==============================] - 3s 46us/step - loss: 0.0223 - accuracy: 0.9795
Epoch 3/20
65040/65040 [==============================] - 3s 46us/step - loss: 0.0134 - accuracy: 0.9867
Epoch 4/20
65040/65040 [==============================] - 3s 46us/step - loss: 0.0094 - accuracy: 0.9897
Epoch 5/20
65040/65040 [==============================] - 3s 47us/step - loss: 0.0072 - accuracy: 0.9924
Epoch 6/20
65040/65040 [==============================] - 3s 45us/step - loss: 0.0058 - accuracy: 0.9936
Epoch 7/20
65040/65040 [==============================] - 3s 44us/step - loss: 0.0050 - accuracy: 0.9944
Epoch 8/20
65040/65040 [==============================] - 3s 44us/step - loss: 0.0041 - accuracy: 0.9953
Epoch 9/20
65040/65040 [==============================] - 3s 44us/step - loss: 0.0039 - accuracy: 0.9958
Epoch 10/20
65040/65040 [==============================

In [356]:
model.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 28, 28)       0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           (None, 28, 28)       0                                            
__________________________________________________________________________________________________
model_19 (Model)                (None, 128)          133504      input_29[0][0]                   
                                                                 input_30[0][0]                   
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 1)            0           model_19[1][0]            

In [357]:
# save resultant model

model.save('model-siamese.h5')

In [358]:
# Create resultant matrix (activation value)
np.random.seed(100)
anomaly_matrix = np.zeros((10,num_classes,500))
for i in range(10):
    te_pairs, te_y = create_test_pairs(i,num_classes,x_test, y_test)
    pred_matrix = create_pred_matrix(num_classes, te_pairs, model)
    anomaly_matrix[i,:,:] = pred_matrix
    

In [360]:
np.save('anomaly_matrix.npy', anomaly_matrix) # save numpy array for zero shot learning